Copyright Preferred Networks, Inc. as contributors to Matlantis contrib project

# ase.io.readがinitial_chargeを持つxyz fileのreadに失敗するbugに対するwork around
aseのversionによっては、initial_chargesがsetされたAtomsにchargesをsetしてase.io.write関数によってxyz fileに出力した際、ase.io.readでそのxyz fileをreadすることができないbugが確認されています。

このscriptでは上記bugの再現とそのwork aroundを紹介します。また、このbug fixは将来的に本家のaseライブラリにも適用され、解消される予定です。

## bugが確認されているversionのaseをinstall

In [1]:
# !pip install ase==3.22.1

## bugの再現
次のcellを実行するとinitial_chargesとchargesがsetされたatomsを保存したxyz fileをreadできないbugを再現できます

In [2]:
import ase
from ase.build import bulk
from ase.calculators.singlepoint import SinglePointCalculator


at = bulk('Si')
initial_charges = [1.0, -1.0]
charges = [-2.0, 2.0]

# initial_chargesをset
at.set_initial_charges(initial_charges)
at.calc = SinglePointCalculator(at, charges=charges)

# chargesをset
at.get_charges()

# xyz fileに出力
ase.io.write('charges1.xyz', at, format='extxyz')

# readに失敗する
r = ase.io.read('charges1.xyz')

RuntimeError: Array initial_charges already present

以下のように、initial chargesをexplicitにsetしない場合もreadの際にinitial chargesがsetされることでbugが再現します。

In [3]:
import ase
from ase.build import bulk
from ase.calculators.singlepoint import SinglePointCalculator


at = bulk('Si')
charges = [-2.0, 2.0]

at.calc = SinglePointCalculator(at, charges=charges)

# chargesをset
at.get_charges()

# xyz fileに出力
ase.io.write('charges2.xyz', at, format='extxyz')
# read
at = ase.io.read('charges2.xyz')
# chargesがinitial_chargesにsetされる
print("initial charges =", at.get_initial_charges())

at.calc = SinglePointCalculator(at, charges=charges)

# chargesをset
at.get_charges()

# xyz fileに出力
ase.io.write('charges2.xyz', at, format='extxyz')
# read
at = ase.io.read('charges2.xyz')

initial charges = [-2.  2.]


RuntimeError: Array initial_charges already present

## xyz fileを保存する前のwork around
xyz fileを保存する前であれば、ase.io.write関数を実行する前にinitial_chargesをresetすることでreadできないbugを回避可能です。

In [4]:
at = bulk('Si')
initial_charges = [1.0, -1.0]
charges = [-2.0, 2.0]

# initial_chargesをset
at.set_initial_charges(initial_charges)
at.calc = SinglePointCalculator(at, charges=charges)

# chargesをset
at.get_charges()

# initial_chargesをreset
at.set_initial_charges()

# xyz fileに出力
ase.io.write('charges3.xyz', at, format='extxyz')

# read
r = ase.io.read('charges3.xyz')

print(r)
# chargesがinitial_chargesにsetされる
print(r.get_initial_charges())

Atoms(symbols='Si2', pbc=True, cell=[[0.0, 2.715, 2.715], [2.715, 0.0, 2.715], [2.715, 2.715, 0.0]], initial_charges=...)
[-2.  2.]


## xyz file保存後にreadする方法
ase_ioにaseのread関数の実装に変更を加えたsource codeを配置しました。ase.io.readで読み込めなくなってしまったxyz fileも以下のread関数で読み込むことができます。

In [5]:
from ase_io import read

In [6]:
r = read('charges1.xyz')
# r = read('charges2.xyz')

print(r)
print(r.get_initial_charges())
print(r.get_charges())

Atoms(symbols='Si2', pbc=True, cell=[[0.0, 2.715, 2.715], [2.715, 0.0, 2.715], [2.715, 2.715, 0.0]], charges=..., initial_charges=..., calculator=SinglePointCalculator(...))
[ 1. -1.]
[-2.  2.]
